In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import re
import math
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from lightgbm import LGBMRegressor, LGBMClassifier
from xgboost import XGBRegressor, XGBClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
import lightgbm as lgb
import xgboost as xgb
import catboost as cab

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold, GridSearchCV
from sklearn.decomposition import LatentDirichletAllocation, NMF, TruncatedSVD
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor, StackingRegressor
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, StackingClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from tqdm import tqdm

import matplotlib.pyplot as plt
from collections import defaultdict, Counter
import warnings
import json 
import pickle
warnings.filterwarnings('ignore')
from sklearn.preprocessing import KBinsDiscretizer

数据读取

In [54]:
# (3606249, 3)
df_train = pd.read_csv('data/sessions_train.csv')
df_train

,prev_items,next_item,locale
0,['B09W9FND7K' 'B09JSPLN1M'],B09M7GY217,DE
1,['B076THCGSG' 'B007MO8IME' 'B08MF65MLV' 'B001B...,B001B4THSA,DE
2,['B0B1LGXWDS' 'B00AZYORS2' 'B0B1LGXWDS' 'B00AZ...,B0767DTG2Q,DE
3,['B09XMTWDVT' 'B0B4MZZ8MB' 'B0B7HZ2GWX' 'B09XM...,B0B4R9NN4B,DE
4,['B09Y5CSL3T' 'B09Y5DPTXN' 'B09FKD61R8'],B0BGVBKWGZ,DE
...,...,...,...
3606244,['B086CYFSKW' 'B0874F9859' 'B086CYFSKW'],B07B5TYD76,IT
3606245,['B09NRZKZ7V' 'B08WJTPV93'],B08L1P4C3D,IT
3606246,['B085JFX7MP' 'B085JGHW8R'],B01MPWVD44,IT
3606247,['B00B0UING2' 'B00B0UING2'],B00D3HYEZ4,IT


In [55]:
df_test_phase1 = pd.read_csv('data/sessions_test_task1.csv')
df_test_phase1

,prev_items,locale
0,['B08V12CT4C' 'B08V1KXBQD' 'B01BVG1XJS' 'B09VC...,DE
1,['B00R9R5ND6' 'B00R9RZ9ZS' 'B00R9RZ9ZS'],DE
2,['B07YSRXJD3' 'B07G7Q5N6G' 'B08C9Q7QVK' 'B07G7...,DE
3,['B08KQBYV43' '3955350843' '3955350843' '39553...,DE
4,['B09FPTCWMC' 'B09FPTQP68' 'B08HMRY8NG' 'B08TB...,DE
...,...,...
316966,['B077SZ2C3Y' 'B0B14M3VZX'],UK
316967,['B08KFHDPY9' 'B0851KTSRZ' 'B08KFHDPY9' 'B0851...,UK
316968,['B07PY1N81F' 'B07Q1Z8SQN' 'B07PY1N81F' 'B07Q1...,UK
316969,['B01MCQMORK' 'B09JYZ325W'],UK


In [56]:
# (316972, 2)
df_test_phase2 = pd.read_csv('data/phase2/sessions_test_task1.csv')
df_test_phase2

,prev_items,locale
0,['B087VLP2RT' 'B09BRQSHYH' 'B099KW4ZLV'],DE
1,['B08XW4W667' 'B096VMCJYF' 'B096VMCJYF'],DE
2,['B09Z4T2GJ3' 'B09Z3FBXMB' 'B0936K9LTJ' 'B09Z4...,DE
3,['B07T6Y2HG7' 'B07T2NBLX9' 'B07Y1G5F3Y'],DE
4,['B0B2DRKZ6X' 'B0B2DRKZ6X' 'B0B2DRKZ6X'],DE
...,...,...
316967,['B078RJX3CC' 'B07GKM97YF'],UK
316968,['B01LX5Y7RG' 'B00M35Y2J0' 'B0BFR9D1Y2' 'B09BB...,UK
316969,['B09HGRXXTM' 'B08VDNCZT9'],UK
316970,['B089CVQ2FS' 'B089CVQ2FS'],UK


In [57]:
# (316972, 2)
df_test_phase2_task2 = pd.read_csv('data/phase2/sessions_test_task2.csv')
df_test_phase2_task2

,prev_items,locale
0,['B07GTS7SWK' 'B07GTS7SWK'],ES
1,['B0B33YWVHR' '849988993X' 'B09QQG85HM' 'B0BJZ...,ES
2,['B08FMPXDTJ' 'B0B4612MTM' 'B0B45YR21M' 'B0B4F...,ES
3,['B07R3W4XQ7' 'B07R3FB5B5'],ES
4,['B09V4KBWPL' 'B09G9FTLPB'],ES
...,...,...
34685,['B089F3YK6X' 'B01BBW9TEI' 'B01BBW9Q46'],IT
34686,['B078XYC42V' 'B09TW31SND'],IT
34687,['B07ZG3V3X2' 'B09PGKF9Y8'],IT
34688,['B0B3JRGZLM' 'B09SHQJ8NV'],IT


In [58]:
def str2list(x):
    x = x.replace('[', '').replace(']', '').replace("'", '').replace('\n', ' ').replace('\r', ' ')
    l = [i for i in x.split() if i]
    return l

In [59]:
# 不到1min (1551057, 11)
products = pd.read_csv('data/products_train.csv')
products

,id,locale,title,price,brand,color,size,model,material,author,desc
0,B005ZSSN10,DE,RED DRAGON Amberjack 3 - Steel Tip 22 Gramm Wo...,30.95,RED DRAGON,NaN,NaN,RDD0089,NaN,NaN,Amberjacks Steel Dartpfeile sind verfügbar in ...
1,B08PRYN6LD,DE,Simply Keto Lower Carb* Schokodrops ohne Zucke...,17.90,Simply Keto,NaN,750 g (1er Pack),NaN,NaN,NaN,🌱 NATÜRLICHE SÜSSE DURCH ERYTHRIT - Wir stelle...
2,B09MBZJ48V,DE,"Sennheiser 508377 PC 5.2 Chat, Stilvolles Mult...",68.89,Sennheiser,Multi-Colour,One size,508377,Kunstleder,NaN,3.5 MM BUCHSE - Kann problemlos an Geräte mit ...
3,B08ZN6F26S,DE,AmyBenton Auto ab 1 2 3 ahre - Baby Aufziehbar...,18.99,Amy & Benton,Animal Car,NaN,2008B,aufziehauto 1 jahr,NaN,【Auto aufziehbar】: Drücken Sie einfach leicht ...
4,B094DGRV7D,DE,PLAYMOBIL - 70522 - Cavaliere mit grauem Pony,7.17,PLAYMOBIL,Nicht Zutreffend.,OneSize,70522,Polypropylen,NaN,Inhalt: 1 Stück
...,...,...,...,...,...,...,...,...,...,...,...
1551052,B09BW5CDRR,IT,Barbie - Playset Gelateria con Bambola con Mac...,20.48,Barbie,NaN,NaN,HCN46,NaN,NaN,DETTAGLI REALISTICI. Basta inserire la pasta m...
1551053,B0050IILBM,IT,"Braun Silk-épil 1 Depilatore Donna, Epilatore ...",22.61,Braun,Pink,NaN,4210201656067,NaN,NaN,Alimentato a corrente per un comodo utilizzo
1551054,B07W4C5W9D,IT,BoxLegend Sacchetti Sottovuoto Vestiti 6 Pezzi...,14.99,BoxLegend,6 Pezzi.,6 Pezzi (2L + 2M + 2S),6186666487608_SML,Polietilene Ppa,NaN,6 Sacchetti in 3 Diverse Misure- Questo set di...
1551055,B012D0HJXA,IT,Trasportino Pratiko Metal - Accessorio da viag...,18.35,MPS,verde,NaN,NaN,Metallo,NaN,TRASPORTINO 48X31.5X33CM


In [60]:
# 1min多一些
df_train['prev_items'] = df_train['prev_items'].apply(lambda x: str2list(x))
df_test_phase1['prev_items'] = df_test_phase1['prev_items'].apply(lambda x: str2list(x))
df_test_phase2['prev_items'] = df_test_phase2['prev_items'].apply(lambda x: str2list(x))

In [61]:
df_test_phase2_task2['prev_items'] = df_test_phase2_task2['prev_items'].apply(lambda x: str2list(x))

In [62]:
df_train['last_item'] = df_train['prev_items'].apply(lambda x: x[-1])
df_test_phase1['last_item'] = df_test_phase1['prev_items'].apply(lambda x: x[-1])
df_test_phase2['last_item'] = df_test_phase2['prev_items'].apply(lambda x: x[-1])

In [63]:
df_test_phase2_task2['last_item'] = df_test_phase2_task2['prev_items'].apply(lambda x: x[-1])

In [64]:
df_train.shape, df_test_phase1.shape, df_test_phase2.shape, df_test_phase2_task2.shape 

((3606249, 4), (316971, 3), (316972, 3), (34690, 3))

In [65]:
loc2id = {
    'DE': 0,
    'JP': 1,
    'UK': 2,
}

In [11]:
# df_train = df_train[df_train['locale'].isin(['DE', 'JP', 'UK'])]
# df_train.shape 

(3272716, 4)

In [12]:
# df_test = df_test[df_test['locale'].isin(['DE', 'JP', 'UK'])]
# df_test.shape 

(316971, 3)

In [13]:
# 后边不用重跑

In [103]:
# train_ids = []
# for _, row in tqdm(df_train.iterrows(), total=len(df_train)):
#     train_ids += row['prev_items']
#     train_ids.append(row['next_item'])
# train_ids = set(train_ids)
# len(train_ids)

100%|██████████| 3272716/3272716 [04:39<00:00, 11691.46it/s]


1326772

In [105]:
# test_ids = []
# for _, row in tqdm(df_test.iterrows(), total=len(df_test)):
#     test_ids += row['prev_items']
# test_ids = set(test_ids)
# len(test_ids)

100%|██████████| 316971/316971 [00:24<00:00, 12737.73it/s]


224336

In [107]:
# len(train_ids & test_ids)

212261

In [108]:
# len(train_ids | test_ids)

1338847

In [109]:
# train_ids = np.array(list(train_ids))
# test_ids = np.array(list(test_ids))
# np.save('data/train_ids.npy', train_ids)
# np.save('data/test_ids.npy', test_ids)

# 召回

召回策略

1. 使用vid关联规则【统计】
    - 计算与最后vid关联性最大的下一个vid；
    - 计算与最后5个vid关联性最大的下一个vid；
    - 计算与历史观看时间最长的vid关联性最大的下一个vid；
2. 使用vid计算词向量相似度

训练集标签中最热门的200个id

In [13]:
# 最热门的200个id
hot200 = df_train['next_item'].value_counts().head(200).index
hot200 = list(hot200)

一阶下一跳

In [76]:
next_item_dict = defaultdict(list)

for _, row in tqdm(df_train.iterrows(), total=len(df_train)):
    prev_items = row['prev_items']
    next_item = row['next_item']
    prev_items_length = len(prev_items)
    if prev_items_length <= 1:
        next_item_dict[prev_items[0]].append(next_item)
    else:
        for i, item in enumerate(prev_items[:-1]):
            next_item_dict[item].append(prev_items[i+1])
        next_item_dict[prev_items[-1]].append(next_item)

100%|██████████| 3272716/3272716 [05:40<00:00, 9618.49it/s] 


In [77]:
for _, row in tqdm(df_test.iterrows(), total=len(df_test)):
    prev_items = row['prev_items']
    prev_items_length = len(prev_items)
    if prev_items_length <= 1:
        continue
    else:
        for i, item in enumerate(prev_items[:-1]):
            next_item_dict[item].append(prev_items[i+1])

100%|██████████| 316971/316971 [00:27<00:00, 11668.89it/s]


In [113]:
pickle.dump(next_item_dict, open('data/next_item_dict.pkl', 'wb'))

In [14]:
next_item_dict = pickle.load(open('data/next_item_dict.pkl', 'rb'))

In [78]:
next_item_map = {}

for item in tqdm(next_item_dict):
    counter = Counter(next_item_dict[item])
    next_item_map[item] = [i[0] for i in counter.most_common(100)]

100%|██████████| 1270206/1270206 [00:13<00:00, 94734.74it/s] 


In [117]:
pickle.dump(next_item_map, open('data/next_item_map.pkl', 'wb'))

In [15]:
next_item_map = pickle.load(open('data/next_item_map.pkl', 'rb'))

一阶下一跳中最热门的200个id

In [79]:
k = []
v = []

for item in next_item_dict:
    k.append(item)
    v.append(next_item_dict[item])
    
df_next = pd.DataFrame({'item': k, 'next_item': v})
df_next = df_next.explode('next_item').reset_index(drop=True)
df_next

,item,next_item
0,B09W9FND7K,B09JSPLN1M
1,B09W9FND7K,B09JSPLN1M
2,B09W9FND7K,B09JSPLN1M
3,B09W9FND7K,B09JSPLN1M
4,B09W9FND7K,B078WW2WN5
...,...,...
15132435,B0BFPLN8FQ,B0BF5GT13X
15132436,B09737CD6H,B08DJ9SQFY
15132437,B084GZ3QZ7,B00CBAW8SE
15132438,B081TXFKS5,B004RN5I6W


In [80]:
top200 = df_next['next_item'].value_counts().index.tolist()[:200]  # 所有下一跳候选集中最热门的200个

In [121]:
pickle.dump(top200, open('data/top200.pkl', 'wb'))

In [16]:
top200 = pickle.load(open('data/top200.pkl', 'rb'))

训练集本地验证结果

In [18]:
# df_train['last_item'] = df_train['prev_items'].apply(lambda x: x[-1])
df_train['next_item_prediction'] = df_train['last_item'].map(next_item_map)
df_train

,prev_items,next_item,locale,last_item,next_item_prediction
0,"[B09W9FND7K, B09JSPLN1M]",B09M7GY217,DE,B09JSPLN1M,"[B09W9FND7K, B09M7GY217, B09YS6S48R, B07KDC7PJ..."
1,"[B076THCGSG, B007MO8IME, B08MF65MLV, B001B4TKA0]",B001B4THSA,DE,B001B4TKA0,"[B001B4THSA, B0001NPYZ2]"
2,"[B0B1LGXWDS, B00AZYORS2, B0B1LGXWDS, B00AZYORS...",B0767DTG2Q,DE,B00AZYORS2,"[B0B1LGXWDS, B0767DTG2Q, B073WWMCDB, B07RB1VFP..."
3,"[B09XMTWDVT, B0B4MZZ8MB, B0B7HZ2GWX, B09XMTWDV...",B0B4R9NN4B,DE,B0B71CHT1L,"[B0B71GSJ2R, B0B71KXCSB, B09XMTWDVT, B0B71JHMT..."
4,"[B09Y5CSL3T, B09Y5DPTXN, B09FKD61R8]",B0BGVBKWGZ,DE,B09FKD61R8,"[B0BGVBKWGZ, B09MRVWQ3B, B09FKDPY2S, B07GZC7JYG]"
...,...,...,...,...,...
3272711,"[B06XK89969, B01NGT5NF4, B00D5Z89C8, B07ZVGCHR...",B07VL2W1DR,UK,B06XK89969,"[B07VL2W1DR, B01NGT5NF4, B07B49NGJV, B07Z4LG5M..."
3272712,"[B076M85W1K, B07L8792Q9, B095RW318L, B095RVVX9T]",B095RQ2LCY,UK,B095RVVX9T,"[B095RW318L, B095RQ2LCY, B09QHQQ26P, B09HT7361..."
3272713,"[B00JQDIQRQ, B001O59QQE]",B088M5YT6Y,UK,B001O59QQE,"[B001O506EK, B001O5499S, B0953S3QX4, B088M5YT6Y]"
3272714,"[B07QMHMLJZ, B07FPYYMC4]",B07PFF814D,UK,B07FPYYMC4,"[B07QMHMLJZ, B07FQC3B57, B0977G35JP, B08MBJF9Q..."


In [ ]:
# TODO: 不一定历史prev_items中的就不会买了，可以去掉限制试试！

In [19]:
train_preds = []
mrr = []

for _, row in tqdm(df_train.iterrows(), total=len(df_train)):
    label = row['next_item']
    pred_orig = row['next_item_prediction']
    pred = pred_orig
    prev_items = row['prev_items']
    if type(pred) == float:  # pred=nan, 训练集的last_item在之间没有统计出来下一跳，此时用最热门代替
        pred = top200[:100]
    else:
        if len(pred_orig) < 100:  # 小于100，从最热门选
            for i in top200:
                # if i not in pred_orig and i not in prev_items:
                if i not in pred_orig:
                    pred.append(i)
                if len(pred) >= 100:
                    break
        else:
            pred = pred[:100]
    try:
        pred_result = pred.index(label)
        mrr.append(1 / (pred_result + 1))
    except:
        mrr.append(0)
    train_preds.append(pred)

np.mean(mrr)

100%|██████████| 3272716/3272716 [18:49<00:00, 2898.34it/s] 


0.4244365572179521

In [29]:
pickle.dump(train_preds, open('./data/train_preds.pkl', 'wb'))

测试集预测结果

In [20]:
# df_test['last_item'] = df_test['prev_items'].apply(lambda x: x[-1])
df_test['next_item_prediction'] = df_test['last_item'].map(next_item_map)
df_test

,prev_items,locale,last_item,next_item_prediction
0,"[B08V12CT4C, B08V1KXBQD, B01BVG1XJS, B09VC5PKN...",DE,B099NQFMG7,"[B099NS1XPG, B08496TCCQ, B01BVG1XJS, B099NR3X6..."
1,"[B00R9R5ND6, B00R9RZ9ZS, B00R9RZ9ZS]",DE,B00R9RZ9ZS,"[B004ZXMV4Q, B00R9R5ND6, B095TQTZXY, B086J6RTT..."
2,"[B07YSRXJD3, B07G7Q5N6G, B08C9Q7QVK, B07G7Q5N6G]",DE,B07G7Q5N6G,"[B08C9Q7QVK, B07G7Q5N6G, B07YSRXJD3, B0B5QNFWJ..."
3,"[B08KQBYV43, 3955350843, 3955350843, 395535086...",DE,3955350843,"[395535086X, 3955350843, B0829LZFT1, 377247695..."
4,"[B09FPTCWMC, B09FPTQP68, B08HMRY8NG, B08TBBQ4B...",DE,B09J945WQR,"[B09J8V18FL, B09J8T6TTH, B09J8SKX9G, B09J8V9RQ..."
...,...,...,...,...
316966,"[B077SZ2C3Y, B0B14M3VZX]",UK,B0B14M3VZX,"[B08X9L5RGD, B07V5FL8G6, B09Y4HKGKT, B09MW64JG..."
316967,"[B08KFHDPY9, B0851KTSRZ, B08KFHDPY9, B0851KTSR...",UK,B081YDH55K,"[B0989BHLSY, B09CPNS7XV, B09895QPQF, B09CPP92Q..."
316968,"[B07PY1N81F, B07Q1Z8SQN, B07PY1N81F, B07Q1Z8SQ...",UK,B09HL11V5B,"[B09HKZBNZH, B09HZSRJWW, B09HL141QC, B07PY1N81..."
316969,"[B01MCQMORK, B09JYZ325W]",UK,B09JYZ325W,"[B07TR5LQSL, B08FB464L7, B09JYZ325W, B08JG8TSC..."


In [21]:
test_preds = []

for _, row in tqdm(df_test.iterrows(), total=len(df_test)):
    pred_orig = row['next_item_prediction']
    pred = pred_orig
    prev_items = row['prev_items']
    if type(pred) == float:  # pred=nan, 测试集的last_item在之间没有统计出来下一跳，此时用最热门代替
        pred = top200[:100]
    else:
        if len(pred_orig) < 100:  # 小于100，从最热门选
            for i in top200:
                if i not in pred_orig:
                    pred.append(i)
                if len(pred) >= 100:
                    break
        else:
            pred = pred[:100]
    test_preds.append(pred)

100%|██████████| 316971/316971 [01:13<00:00, 4291.98it/s]


In [22]:
len(test_preds)

316971

In [66]:
pickle.dump(test_preds, open('./data/test_preds.pkl', 'wb'))

保存结果

In [23]:
df_test['next_item_prediction'] = test_preds
df_test['next_item_prediction'].apply(len).describe()

count    316971.0
mean        100.0
std           0.0
min         100.0
25%         100.0
50%         100.0
75%         100.0
max         100.0
Name: next_item_prediction, dtype: float64

In [24]:
df_test[['locale', 'next_item_prediction']].to_parquet('output/submission_task1_rule_next_one_allow_prevs.parquet', engine='pyarrow')

# 数据处理

产品表处理

In [21]:
# TODO: 部分ID不在产品表里？？没有！

In [2]:
products = pd.read_csv('data/products_train.csv')
products

,id,locale,title,price,brand,color,size,model,material,author,desc
0,B005ZSSN10,DE,RED DRAGON Amberjack 3 - Steel Tip 22 Gramm Wo...,30.95,RED DRAGON,NaN,NaN,RDD0089,NaN,NaN,Amberjacks Steel Dartpfeile sind verfügbar in ...
1,B08PRYN6LD,DE,Simply Keto Lower Carb* Schokodrops ohne Zucke...,17.90,Simply Keto,NaN,750 g (1er Pack),NaN,NaN,NaN,🌱 NATÜRLICHE SÜSSE DURCH ERYTHRIT - Wir stelle...
2,B09MBZJ48V,DE,"Sennheiser 508377 PC 5.2 Chat, Stilvolles Mult...",68.89,Sennheiser,Multi-Colour,One size,508377,Kunstleder,NaN,3.5 MM BUCHSE - Kann problemlos an Geräte mit ...
3,B08ZN6F26S,DE,AmyBenton Auto ab 1 2 3 ahre - Baby Aufziehbar...,18.99,Amy & Benton,Animal Car,NaN,2008B,aufziehauto 1 jahr,NaN,【Auto aufziehbar】: Drücken Sie einfach leicht ...
4,B094DGRV7D,DE,PLAYMOBIL - 70522 - Cavaliere mit grauem Pony,7.17,PLAYMOBIL,Nicht Zutreffend.,OneSize,70522,Polypropylen,NaN,Inhalt: 1 Stück
...,...,...,...,...,...,...,...,...,...,...,...
1551052,B09BW5CDRR,IT,Barbie - Playset Gelateria con Bambola con Mac...,20.48,Barbie,NaN,NaN,HCN46,NaN,NaN,DETTAGLI REALISTICI. Basta inserire la pasta m...
1551053,B0050IILBM,IT,"Braun Silk-épil 1 Depilatore Donna, Epilatore ...",22.61,Braun,Pink,NaN,4210201656067,NaN,NaN,Alimentato a corrente per un comodo utilizzo
1551054,B07W4C5W9D,IT,BoxLegend Sacchetti Sottovuoto Vestiti 6 Pezzi...,14.99,BoxLegend,6 Pezzi.,6 Pezzi (2L + 2M + 2S),6186666487608_SML,Polietilene Ppa,NaN,6 Sacchetti in 3 Diverse Misure- Questo set di...
1551055,B012D0HJXA,IT,Trasportino Pratiko Metal - Accessorio da viag...,18.35,MPS,verde,NaN,NaN,Metallo,NaN,TRASPORTINO 48X31.5X33CM


In [66]:
products.drop_duplicates(subset=['id'], keep='first', inplace=True)
products.shape 

(1410675, 11)

In [67]:
cat_features = ['brand', 'color', 'size', 'model', 'material', 'author']
num_features = ['price']

In [68]:
unique_categories_list = []
for f in cat_features:
    products['encode_' + f], unique_categories = pd.factorize(products[f], na_sentinel=-1)  # nan->-1, +1变成0
    products['encode_' + f] += 1
    print(f, len(unique_categories), (products['encode_' + f] == 0).sum())
    unique_categories_list.append(unique_categories.insert(0, '0'))  # 插入一个0，作为空值

brand 177189 19224
color 203260 402091
size 218060 580806
model 524101 748606
material 45568 659226
author 30835 1338577


In [69]:
# le_list = []
# for f in cat_features:
#     le = LabelEncoder()
#     products['encode_' + f] = le.fit_transform(products[f])
#     le_list.append(le)

In [70]:
products.shape 

(1410675, 17)

In [71]:
dense_bins = 100
add_cat_features = []
for f in num_features:
    discretizer = KBinsDiscretizer(n_bins=dense_bins, encode='ordinal', strategy='kmeans')  # 等频quantile，等宽uniform
    products['encode_' + f] = discretizer.fit_transform(np.array(products[f].tolist()).reshape(-1, 1))
    add_cat_features.append('encode_' + f)

In [72]:
for fe in add_cat_features:
    products[fe] = products[fe].astype('int64')

In [73]:
for fe in num_features:
    products[fe] = products[fe].astype('float32')

In [74]:
features = [f for f in products.columns if f not in ['title', 'desc'] + cat_features]
len(features)

10

In [75]:
cat_features

['brand', 'color', 'size', 'model', 'material', 'author']

In [76]:
features

['id',
 'locale',
 'price',
 'encode_brand',
 'encode_color',
 'encode_size',
 'encode_model',
 'encode_material',
 'encode_author',
 'encode_price']

In [147]:
products_encoded = pd.read_csv('./data/products_encoded.csv')

In [151]:
products_encoded_newfeature = pd.read_csv('./data/products_encoded_newfeature.csv')

In [152]:
products_encoded_newfeature.shape 

(1410675, 10)

In [153]:
products_encoded_newfeature.head()

,id,locale,price,encode_brand,encode_color,encode_size,encode_model,encode_material,encode_author,encode_price
0,0,0,30.95,112134,203260,218060,426630,45568,30835,0
1,1,0,17.90,124505,203260,128007,524101,45568,30835,0
2,2,0,68.89,122979,114264,170270,145013,15566,30835,0
3,3,0,18.99,9834,46931,218060,67408,29357,30835,0
4,4,0,7.17,105135,117844,170305,174527,23064,30835,0


In [149]:
products_encoded.shape 

(1410675, 14)

In [161]:
product2id = json.load(open('data/product2id.json', 'r'))
id2product = json.load(open('data/id2product.json', 'r'))
id2product = {int(k): v for k, v in id2product.items()}

In [162]:
products_encoded['id'] = products_encoded['id'].map(product2id)

In [163]:
loc2id = {'DE': 0, 'JP': 1, 'UK': 2, 'ES': 3, 'FR': 4, 'IT': 5}

In [164]:
products_encoded['locale'] = products_encoded['locale'].map(loc2id)

In [167]:
products_encoded.to_csv('./data/products_encoded_phase2_V3.csv', index=False)

In [168]:
products_encoded.shape 

(1410675, 10)

测试集跟产品表连表

In [107]:
df_test_phase2 = df_test_phase2_task2

In [108]:
df_test_phase2.columns

Index(['prev_items', 'locale', 'last_item'], dtype='object')

In [109]:
df_test_exploded = df_test_phase2[['prev_items', 'locale', 'last_item']].explode('prev_items')

In [110]:
df_test_exploded.index 

Int64Index([    0,     0,     1,     1,     1,     1,     1,     1,     2,
                2,
            ...
            34685, 34686, 34686, 34687, 34687, 34688, 34688, 34689, 34689,
            34689],
           dtype='int64', length=132551)

In [111]:
df_test_exploded['index'] = df_test_exploded.index 

In [112]:
df_test_exploded = df_test_exploded.rename(columns={'prev_items': 'id'})

In [113]:
df_test_exploded

,id,locale,last_item,index
0,B07GTS7SWK,ES,B07GTS7SWK,0
0,B07GTS7SWK,ES,B07GTS7SWK,0
1,B0B33YWVHR,ES,B09K7TDY1H,1
1,849988993X,ES,B09K7TDY1H,1
1,B09QQG85HM,ES,B09K7TDY1H,1
...,...,...,...,...
34688,B0B3JRGZLM,IT,B09SHQJ8NV,34688
34688,B09SHQJ8NV,IT,B09SHQJ8NV,34688
34689,B08ZHKGYXR,IT,B0882QYG73,34689
34689,B07Z8PZTGL,IT,B0882QYG73,34689


In [114]:
# products在同一个地区的ID是不一样的，但是不同地区的ID可能会一样
df_test_exploded_fea = df_test_exploded.merge(products, on='id', how='left')
df_test_exploded_fea.head()

,id,locale_x,last_item,index,locale_y,title,price,brand,color,size,...,material,author,desc,encode_brand,encode_color,encode_size,encode_model,encode_material,encode_author,encode_price
0,B07GTS7SWK,ES,B07GTS7SWK,0,DE,HP 303 (3YM92AE) Original Druckerpatronen Mult...,34.52,HP,"Schwarz, Cyan, Magenta, Gelb",Standard,...,NaN,NaN,"Kompatibel mit: HP ENVY Photo 6230, HP ENVY Ph...",1412,2132,376,8976,0,0,0
1,B07GTS7SWK,ES,B07GTS7SWK,0,DE,HP 303 (3YM92AE) Original Druckerpatronen Mult...,34.52,HP,"Schwarz, Cyan, Magenta, Gelb",Standard,...,NaN,NaN,"Kompatibel mit: HP ENVY Photo 6230, HP ENVY Ph...",1412,2132,376,8976,0,0,0
2,B0B33YWVHR,ES,B09K7TDY1H,1,ES,Todo arde,4.74,EDICIONES B,NaN,NaN,...,NaN,"Gómez-Jurado, Juan",NaN,168665,0,0,0,0,29223,0
3,849988993X,ES,B09K7TDY1H,1,ES,Cómo el organismo esculpe el cerebro,17.10,Editorial Kairós SA,NaN,NaN,...,NaN,"Castellanos, Nazareth",NaN,169365,0,0,0,0,29354,0
4,B09QQG85HM,ES,B09K7TDY1H,1,DE,Xiaomi Redmi Buds 3 Lite kabellose Bluetooth 5...,20.49,Xiaomi,Black,NaN,...,silikon,NaN,【Bis zu 18 Stunden Akkulaufzeit】Xiaomi kopfhör...,527,79,0,5795,152,0,0


In [115]:
df_test_exploded_fea.shape 

(132551, 21)

训练集和产品表连表

In [85]:
df_train.columns

Index(['prev_items', 'next_item', 'locale', 'last_item'], dtype='object')

In [86]:
df_train_exploded = df_train[['prev_items', 'locale', 'last_item', 'next_item']].explode('prev_items')

In [87]:
df_train_exploded['index'] = df_train_exploded.index 

In [88]:
df_train_exploded = df_train_exploded.rename(columns={'prev_items': 'id'})

In [89]:
# products在同一个地区的ID是不一样的，但是不同地区的ID可能会一样
df_train_exploded_fea = df_train_exploded.merge(products, on='id', how='left')
df_train_exploded_fea.head()

,id,locale_x,last_item,next_item,index,locale_y,title,price,brand,color,...,material,author,desc,encode_brand,encode_color,encode_size,encode_model,encode_material,encode_author,encode_price
0,B09W9FND7K,DE,B09JSPLN1M,B09M7GY217,0,DE,OREiN LED Einbaustrahler 230V Dimmbar & IP65 L...,39.990002,OREiN,"6*ip65 (3000k,dimmbar)",...,塑料,NaN,【Angenehmes Licht】Dieser Einbaustrahler LED Fl...,29880,48058,20832,0,333,0,0
1,B09JSPLN1M,DE,B09JSPLN1M,B09M7GY217,0,DE,OREiN LED Spot LED Einbaustrahler 230V IP65 LE...,36.990002,OREiN,6*ip65(3000k),...,Kunststoff,NaN,【Angenehmes Licht】Dieser Einbaustrahler LED Fl...,29880,18296,20832,0,14,0,0
2,B076THCGSG,DE,B001B4TKA0,B001B4THSA,1,DE,"VIETSCHI Profi DIN Plus 12,5L weiß - doppeldec...",39.799999,Vietschi,NaN,...,NaN,NaN,"✅ Einfache Verarbeitung - Egal ob Streichen, R...",18500,0,11071,0,0,0,0
3,B007MO8IME,DE,B001B4TKA0,B001B4THSA,1,DE,"Caparol Capa DIN 12,500 L Wandfarben",40.270000,Caparol,Weiss,...,Dispersionsfarbe,NaN,Caparol,30369,205,11071,38392,5050,0,0
4,B08MF65MLV,DE,B001B4TKA0,B001B4THSA,1,DE,Metzler Edelstahl Türklingel in Anthrazit - Tü...,29.990000,Metzler,Anthrazit,...,Edelstahl V2A,NaN,Der Anschluss erfolgt an herkömmlichen Klingel...,3245,222,1646,0,548,0,0


In [90]:
df_train_exploded_fea.shape 

(15306183, 22)

测试集序列特征抽取

In [91]:
def mymode(x):
    return int(x.mode()[0])

def quantile75(x):
    return x.quantile(0.75)

def quantile25(x):
    return x.quantile(0.25)

In [92]:
agg = {
    'id': ['nunique', 'count'],
    'encode_brand' : ['nunique', 'count', mymode],
    'encode_color' : ['nunique', 'count', mymode],
    'encode_size' : ['nunique', 'count', mymode],
    'encode_model' : ['nunique', 'count', mymode],
    'encode_material' : ['nunique', 'count', mymode],
    'encode_author' : ['nunique', 'count', mymode],
    'price' : ['mean', 'std', 'min', 'max', 'sum', np.ptp, quantile75, quantile25],
}

In [116]:
df_test_seqs_feas = df_test_exploded_fea.groupby('index').agg(agg)

In [117]:
df_test_seqs_feas.columns = pd.Index([e[0] + e[1].upper() for e in df_test_seqs_feas.columns.tolist()])

In [118]:
df_test_seqs_feas.head()

,idNUNIQUE,idCOUNT,encode_brandNUNIQUE,encode_brandCOUNT,encode_brandMYMODE,encode_colorNUNIQUE,encode_colorCOUNT,encode_colorMYMODE,encode_sizeNUNIQUE,encode_sizeCOUNT,...,encode_authorCOUNT,encode_authorMYMODE,priceMEAN,priceSTD,priceMIN,priceMAX,priceSUM,pricePTP,priceQUANTILE75,priceQUANTILE25
index,,,,,,,,,,,,,,,,,,,,,
0,1,2,1,2,1412,1,2,2132,1,2,...,2,0,34.520000,0.000000,34.52,34.520000,69.040001,0.000000,34.520000,34.520000
1,6,6,4,6,527,5,6,0,2,6,...,6,0,18.708334,9.589059,4.74,29.980000,112.250000,25.240000,26.115000,13.237500
2,4,6,1,6,527,3,6,192697,4,6,...,6,0,142.183334,28.716279,121.00,199.000000,853.099976,78.000000,138.949997,127.599998
3,2,2,1,2,405,1,2,193945,2,2,...,2,0,59.075001,48.203472,24.99,93.160004,118.150002,68.170006,76.117500,42.032501
4,2,2,1,2,962,2,2,11921,2,2,...,2,0,922.000000,66.468037,875.00,969.000000,1844.000000,94.000000,945.500000,898.500000


In [119]:
df_test_seqs_feas.shape 

(34690, 28)

In [170]:
df_test_seqs_feas.columns

Index(['idNUNIQUE', 'idCOUNT', 'encode_brandNUNIQUE', 'encode_brandCOUNT',
       'encode_brandMYMODE', 'encode_colorNUNIQUE', 'encode_colorCOUNT',
       'encode_colorMYMODE', 'encode_sizeNUNIQUE', 'encode_sizeCOUNT',
       'encode_sizeMYMODE', 'encode_modelNUNIQUE', 'encode_modelCOUNT',
       'encode_modelMYMODE', 'encode_materialNUNIQUE', 'encode_materialCOUNT',
       'encode_materialMYMODE', 'encode_authorNUNIQUE', 'encode_authorCOUNT',
       'encode_authorMYMODE', 'priceMEAN', 'priceSTD', 'priceMIN', 'priceMAX',
       'priceSUM', 'pricePTP', 'priceQUANTILE75', 'priceQUANTILE25',
       'encode_priceMEAN', 'encode_priceSTD', 'encode_priceMIN',
       'encode_priceMAX', 'encode_priceSUM', 'encode_pricePTP',
       'encode_priceQUANTILE75', 'encode_priceQUANTILE25'],
      dtype='object')

In [176]:
[f for f in df_test_seqs_feas.columns if 'MODE' not in f and ('NUNIQUE' in f or 'COUNT' in f or 'encode' in f)]

['idNUNIQUE',
 'idCOUNT',
 'encode_brandNUNIQUE',
 'encode_brandCOUNT',
 'encode_colorNUNIQUE',
 'encode_colorCOUNT',
 'encode_sizeNUNIQUE',
 'encode_sizeCOUNT',
 'encode_modelNUNIQUE',
 'encode_modelCOUNT',
 'encode_materialNUNIQUE',
 'encode_materialCOUNT',
 'encode_authorNUNIQUE',
 'encode_authorCOUNT',
 'encode_priceMEAN',
 'encode_priceSTD',
 'encode_priceMIN',
 'encode_priceMAX',
 'encode_priceSUM',
 'encode_pricePTP',
 'encode_priceQUANTILE75',
 'encode_priceQUANTILE25']

In [172]:
[f for f in df_test_seqs_feas.columns if 'MODE' in f]

['encode_brandMYMODE',
 'encode_colorMYMODE',
 'encode_sizeMYMODE',
 'encode_modelMYMODE',
 'encode_materialMYMODE',
 'encode_authorMYMODE']

In [177]:
[f for f in df_test_seqs_feas.columns if f not in ([f for f in df_test_seqs_feas.columns if 'MODE' not in f and ('NUNIQUE' in f or 'COUNT' in f or 'encode' in f)]) and f not in ([f for f in df_test_seqs_feas.columns if 'MODE' in f])]

['priceMEAN',
 'priceSTD',
 'priceMIN',
 'priceMAX',
 'priceSUM',
 'pricePTP',
 'priceQUANTILE75',
 'priceQUANTILE25']

In [121]:
seqs_cat_feas = [f for f in df_test_seqs_feas.columns if 'NUNIQUE' in f or 'COUNT' in f or 'MODE' in f]

In [122]:
seqs_num_feas = [f for f in df_test_seqs_feas.columns if f not in seqs_cat_feas]

In [123]:
len(seqs_cat_feas), len(seqs_num_feas)

(20, 8)

In [124]:
dense_bins = 100
add_cat_features = []
for f in seqs_num_feas:
    discretizer = KBinsDiscretizer(n_bins=dense_bins, encode='ordinal', strategy='kmeans')  # 等频quantile，等宽uniform
    df_test_seqs_feas['encode_' + f] = discretizer.fit_transform(np.array(df_test_seqs_feas[f].tolist()).reshape(-1, 1))
    add_cat_features.append('encode_' + f)

In [ ]:
for fe in add_cat_features:
    df_test_seqs_feas[fe] = df_test_seqs_feas[fe].astype('int64')

In [205]:
df_test_seqs_feas.shape 

(34690, 36)

训练集序列特征抽取

In [128]:
df_train_seqs_feas = df_train_exploded_fea.groupby('index').agg(agg)

In [129]:
df_train_seqs_feas.columns = pd.Index([e[0] + e[1].upper() for e in df_train_seqs_feas.columns.tolist()])

In [130]:
df_train_seqs_feas.head()

,idNUNIQUE,idCOUNT,encode_brandNUNIQUE,encode_brandCOUNT,encode_brandMYMODE,encode_colorNUNIQUE,encode_colorCOUNT,encode_colorMYMODE,encode_sizeNUNIQUE,encode_sizeCOUNT,...,encode_authorCOUNT,encode_authorMYMODE,priceMEAN,priceSTD,priceMIN,priceMAX,priceSUM,pricePTP,priceQUANTILE75,priceQUANTILE25
index,,,,,,,,,,,,,,,,,,,,,
0,2,2,1,2,29880,2,2,18296,1,2,...,2,0,38.490002,2.121320,36.990002,39.990002,76.980003,3.000000,39.240002,37.740002
1,4,4,4,4,1026,4,4,0,3,4,...,4,0,31.762501,10.929280,16.990000,40.270000,127.050003,23.280001,39.917500,26.740000
2,2,9,1,9,14524,1,9,0,1,9,...,9,0,60.873337,8.427471,51.990002,67.980003,547.860046,15.990002,67.980003,51.990002
3,4,7,1,7,10709,4,7,29,1,7,...,7,0,14.418571,0.534522,13.990000,14.990000,100.930000,1.000000,14.990000,13.990000
4,3,3,2,3,761,2,3,43,3,3,...,3,0,23.656668,11.590227,12.990000,35.990002,70.970001,23.000002,28.990002,17.490000


In [131]:
df_train_seqs_feas.shape 

(3606249, 28)

In [132]:
seqs_cat_feas = [f for f in df_train_seqs_feas.columns if 'NUNIQUE' in f or 'COUNT' in f or 'MODE' in f]

In [133]:
seqs_num_feas = [f for f in df_train_seqs_feas.columns if f not in seqs_cat_feas]

In [134]:
len(seqs_cat_feas), len(seqs_num_feas)

(20, 8)

In [135]:
dense_bins = 100
add_cat_features = []
for f in seqs_num_feas:
    discretizer = KBinsDiscretizer(n_bins=dense_bins, encode='ordinal', strategy='kmeans')  # 等频quantile，等宽uniform
    df_train_seqs_feas['encode_' + f] = discretizer.fit_transform(np.array(df_train_seqs_feas[f].tolist()).reshape(-1, 1))
    add_cat_features.append('encode_' + f)

In [ ]:
for fe in add_cat_features:
    df_train_seqs_feas[fe] = df_train_seqs_feas[fe].astype('int64')

In [138]:
add_cat_features

['encode_priceMEAN',
 'encode_priceSTD',
 'encode_priceMIN',
 'encode_priceMAX',
 'encode_priceSUM',
 'encode_pricePTP',
 'encode_priceQUANTILE75',
 'encode_priceQUANTILE25']

In [137]:
df_train_seqs_feas.shape 

(3606249, 36)

In [136]:
df_train_seqs_feas.to_csv('data/df_train_seqs_feas_all_phase2_V3.csv', index=True)

In [139]:
df_train_seqs_feas.columns

Index(['idNUNIQUE', 'idCOUNT', 'encode_brandNUNIQUE', 'encode_brandCOUNT',
       'encode_brandMYMODE', 'encode_colorNUNIQUE', 'encode_colorCOUNT',
       'encode_colorMYMODE', 'encode_sizeNUNIQUE', 'encode_sizeCOUNT',
       'encode_sizeMYMODE', 'encode_modelNUNIQUE', 'encode_modelCOUNT',
       'encode_modelMYMODE', 'encode_materialNUNIQUE', 'encode_materialCOUNT',
       'encode_materialMYMODE', 'encode_authorNUNIQUE', 'encode_authorCOUNT',
       'encode_authorMYMODE', 'priceMEAN', 'priceSTD', 'priceMIN', 'priceMAX',
       'priceSUM', 'pricePTP', 'priceQUANTILE75', 'priceQUANTILE25',
       'encode_priceMEAN', 'encode_priceSTD', 'encode_priceMIN',
       'encode_priceMAX', 'encode_priceSUM', 'encode_pricePTP',
       'encode_priceQUANTILE75', 'encode_priceQUANTILE25'],
      dtype='object')

In [142]:
mode_features = [f for f in df_train_seqs_feas.columns if 'MODE' in f]

In [143]:
mode_features

['encode_brandMYMODE',
 'encode_colorMYMODE',
 'encode_sizeMYMODE',
 'encode_modelMYMODE',
 'encode_materialMYMODE',
 'encode_authorMYMODE']

In [137]:
df_train_seqs_feas.to_csv('data/df_train_seqs_feas_all_phase2_V3.csv', index=True)

In [138]:
df_train_seqs_feas.columns 

Index(['idNUNIQUE', 'idCOUNT', 'encode_brandNUNIQUE', 'encode_brandCOUNT',
       'encode_brandMYMODE', 'encode_colorNUNIQUE', 'encode_colorCOUNT',
       'encode_colorMYMODE', 'encode_sizeNUNIQUE', 'encode_sizeCOUNT',
       'encode_sizeMYMODE', 'encode_modelNUNIQUE', 'encode_modelCOUNT',
       'encode_modelMYMODE', 'encode_materialNUNIQUE', 'encode_materialCOUNT',
       'encode_materialMYMODE', 'encode_authorNUNIQUE', 'encode_authorCOUNT',
       'encode_authorMYMODE', 'priceMEAN', 'priceSTD', 'priceMIN', 'priceMAX',
       'priceSUM', 'pricePTP', 'priceQUANTILE75', 'priceQUANTILE25'],
      dtype='object')

In [140]:
df_train_encoded = pd.read_csv('data/df_train_encoded.csv')

In [144]:
df_train_seqs_feas[mode_features].head()

,encode_brandMYMODE,encode_colorMYMODE,encode_sizeMYMODE,encode_modelMYMODE,encode_materialMYMODE,encode_authorMYMODE
index,,,,,,
0,101601,37318,163936,524101,15665,30835
1,27926,47134,23154,524101,9922,30835
2,141835,203260,121077,430479,45568,30835
3,4719,60159,218060,501311,15566,30835
4,14269,91302,28876,524101,18264,30835


In [141]:
df_train_encoded.head()

,prev_items,next_item,locale,last_item
0,"[265193, 83226]",387776,0,83226
1,"[38788, 85634, 4132, 71046]",335301,0,71046
2,"[191882, 516876, 191882, 516876, 191882, 19188...",90141,0,516876
3,"[136959, 261145, 31496, 136959, 261145, 31496,...",214540,0,469511
4,"[291068, 410614, 4219]",338089,0,4219


In [145]:
product2id = json.load(open('data/product2id.json', 'r'))
id2product = json.load(open('data/id2product.json', 'r'))
id2product = {int(k): v for k, v in id2product.items()}

# 排序

In [55]:
# def get_feature(index, train_seq, seq_featurs_index):
#     train_fea = []

#     label = train_seq['next_item'].iloc[0]
#     last_item = df_train['last_item'].iloc[0]
#     if last_item in next_item_map:
#         next_item_prediction = next_item_map[last_item]
#     else:
#         next_item_prediction = []
#     # print(next_item_prediction)
#     prev_items = list(train_seq['id'])
#     locale = train_seq['locale'].iloc[0]
#     loc_id = loc2id[locale]  # 地区ID

#     candidate_id = []
#     candidate_id = next_item_prediction  # 1-hop预测的结果
#     if len(next_item_prediction) == 0:  # pred=nan, 训练集的last_item在之间没有统计出来下一跳，此时用最热门代替
#         candidate_id = top200[:100]
#     else:
#         if len(candidate_id) < 100:  # 小于100，从最热门选
#             for i in top200:
#                 if i not in candidate_id and i not in prev_items:
#                     candidate_id.append(i)
#                 if len(candidate_id) >= 100:
#                     break
#         else:
#             candidate_id = candidate_id[:100]
#     # 其他召回策略，比如hop200, w2v等
#     # 另外，这个位置的写法之后改成芒果TV的做法，比如nan的时候直接top200也可能会有历史重复的物品，这样不太好！！！
#     # 目前先验证这个固定的召回+排序之后会不会编号
    
#     # 构造一些序列特征 根据历史的seq，就是seq_featurs_index
    
#     # TODO: 加入item本身的特征 通过连表操作
#     # TODO: 候选集太多，数据量太大了，sample 3个负样本算了
#     if label not in candidate_id:
#         sample = candidate_id[:5]
#         #sample = np.random.choice(candidate_id, size=3, replace=False).tolist()
#     else:
#         candidate_id.remove(label)
#         sample = candidate_id[:5]
#         #sample = np.random.choice(candidate_id, size=3, replace=False).tolist()
#     sample.append(label)
#     for vid in sample:
#         if vid == label:
#             lbl = 1
#         else:
#             lbl = 0
#         # print(vid, locale)
#         try:
#             tmp = products.loc[(vid, locale)]  # 产品特征
#             item_price = tmp.price
#             item_brand = tmp.brand 
#             item_color = tmp.color 
#             item_size = tmp.size 
#             item_model = tmp.model 
#             item_material = tmp.material 
#             item_author = tmp.author 
#         except:
#             continue
        
#         feat = [
#             # index,  这个比赛没有用户ID这一个特征！
#             int(vid in prev_items),  # 候选的vid是否出现在历史购买的序列中
#             int(vid in next_item_prediction),  # 候选的vid是否出现在上次购买产品的下一跳序列中
#             loc_id,
#             item_price,
#             item_brand,
#             item_color,
#             item_size,
#             item_model,
#             item_material,
#             item_author,
#             # 其他特征
#         ]
#         feat += seq_featurs_index
#         # try:
#         #     feat.append(vid_w2v.wv.similarity(vid, last1_vid))
#         # except:
#         #     feat.append(0)
#         feat.append(lbl)
#         train_fea.append(feat)
#     return train_fea

In [56]:
# a = get_feature(0, df_train_exploded_fea[:2], df_train_seqs_feas.loc[0].values.tolist())

训练特征

In [57]:
def get_train_feature(index, train_seq, seq_featurs_index, products_feas):
    train_fea = []

    label = train_seq['next_item'].iloc[0]
    prev_items = list(train_seq['id'])
    locale = train_seq['locale'].iloc[0]
    loc_id = loc2id[locale]  # 地区ID

    candidate_id = train_preds[index]
    # 其他召回策略，比如hop200, w2v等
    # 另外，这个位置的写法之后改成芒果TV的做法，比如nan的时候直接top200也可能会有历史重复的物品，这样不太好！！！
    # 目前先验证这个固定的召回+排序之后会不会编号
    
    # 构造一些序列特征 根据历史的seq，就是seq_featurs_index
    
    # TODO: 候选集太多，数据量太大了，sample 5个负样本算了
    if label in candidate_id:
        candidate_id.remove(label)
    
    # 5个负样本
    cnt = 0
    for vid in candidate_id:
        try:
            pro_feas_index = products_feas.loc[(vid, locale)].values.tolist()  # 产品特征
            cnt += 1
            if cnt > 5:
                break
        except:
            continue
        
        feat = [
            # index,  这个比赛没有用户ID这一个特征！
            int(vid in prev_items),  # 候选的vid是否出现在历史购买的序列中
            loc_id,
            # 其他特征
        ]
        feat += pro_feas_index
        feat += seq_featurs_index
        # try:
        #     feat.append(vid_w2v.wv.similarity(vid, last1_vid))
        # except:
        #     feat.append(0)
        feat.append(0)  # label
        train_fea.append(feat)
    
    # 正样本
    vid = label
    try:
        pro_feas_index = products_feas.loc[(vid, locale)].values.tolist()  # 产品特征
    except:
        print(vid, locale)
        
    feat = [
        # index,  这个比赛没有用户ID这一个特征！
        int(vid in prev_items),  # 候选的vid是否出现在历史购买的序列中
        loc_id,
        # 其他特征
    ]
    feat += pro_feas_index
    feat += seq_featurs_index
    # try:
    #     feat.append(vid_w2v.wv.similarity(vid, last1_vid))
    # except:
    #     feat.append(0)
    feat.append(1)  # label
    train_fea.append(feat)
    return train_fea

In [74]:
train_fea = []
df_train_len = df_train_exploded_fea['index'].nunique()
for ind, train_seq_index in tqdm(df_train_exploded_fea.groupby('index'), total=df_train_len):
    train_fea_in = get_train_feature(ind, train_seq_index, df_train_seqs_feas.loc[ind].values.tolist(), products_feas=products_use)
    train_fea += train_fea_in

100%|██████████| 3272716/3272716 [6:13:14<00:00, 146.14it/s]   


In [75]:
np.save('./data/train_features.npy', train_fea)

In [58]:
train_fea = np.load('./data/train_features.npy')

In [59]:
train_fea.shape 

(19636288, 41)

In [60]:
train_fea = pd.DataFrame(train_fea)
train_fea = train_fea.fillna(0)
print(train_fea.iloc[:, -1].value_counts(normalize=True))

0.0    0.833333
1.0    0.166667
Name: 40, dtype: float64


测试特征

In [61]:
def get_test_feature(index, test_seq, seq_featurs_index, products_feas):
    test_fea = []

    prev_items = list(test_seq['id'])
    locale = test_seq['locale'].iloc[0]
    loc_id = loc2id[locale]  # 地区ID

    candidate_id = test_preds[index]
    # 其他召回策略，比如hop200, w2v等
    # 另外，这个位置的写法之后改成芒果TV的做法，比如nan的时候直接top200也可能会有历史重复的物品，这样不太好！！！
    # 目前先验证这个固定的召回+排序之后会不会编号
    
    # 构造一些序列特征 根据历史的seq，就是seq_featurs_index
    
    cnt = 0
    for vid in candidate_id:
        try:
            pro_feas_index = products_feas.loc[(vid, locale)].values.tolist()  # 产品特征
            cnt += 1
            if cnt > 5:
                break
        except:
            continue  # TODO: continue的话，概率怎么跟vid对应
        
        feat = [
            # index,  这个比赛没有用户ID这一个特征！
            vid,
            int(vid in prev_items),  # 候选的vid是否出现在历史购买的序列中
            loc_id,
            # 其他特征
        ]
        feat += pro_feas_index
        feat += seq_featurs_index
        # try:
        #     feat.append(vid_w2v.wv.similarity(vid, last1_vid))
        # except:
        #     feat.append(0)
        # feat.append(0)  # label
        test_fea.append(feat)
    return test_fea

In [68]:
test_fea = []
df_test_len = df_test_exploded_fea['index'].nunique()
for ind, test_seq_index in tqdm(df_test_exploded_fea.groupby('index'), total=df_test_len):
    test_fea_in = get_test_feature(ind, test_seq_index, df_test_seqs_feas.loc[ind].values.tolist(), products_feas=products_use)
    test_fea += test_fea_in

100%|██████████| 316971/316971 [25:45<00:00, 205.08it/s]


In [75]:
np.save('./data/test_features.npy', test_fea)

In [62]:
test_fea = np.load('./data/test_features.npy')

In [63]:
test_fea = pd.DataFrame(test_fea)
test_fea = test_fea.fillna(0)

In [64]:
for col in test_fea.columns[1:]:
    test_fea[col] = test_fea[col].astype('float64')

In [65]:
test_fea.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,B099NS1XPG,0.0,0.0,39.99,21236.0,152211.0,213319.0,161238.0,44782.0,29259.0,...,163.333333,24.436684,126.0,195.0,1960.0,173.583333,34.057327,120.0,210.0,2083.0
1,B08496TCCQ,1.0,0.0,40000000.07,21236.0,133489.0,213319.0,161226.0,44782.0,29259.0,...,163.333333,24.436684,126.0,195.0,1960.0,173.583333,34.057327,120.0,210.0,2083.0
2,B01BVG1XJS,1.0,0.0,19.95,21236.0,152211.0,90884.0,161227.0,15295.0,29259.0,...,163.333333,24.436684,126.0,195.0,1960.0,173.583333,34.057327,120.0,210.0,2083.0
3,B099NR3X6D,0.0,0.0,32.99,21236.0,133489.0,213319.0,161237.0,44782.0,29259.0,...,163.333333,24.436684,126.0,195.0,1960.0,173.583333,34.057327,120.0,210.0,2083.0
4,B09WDSH4CD,0.0,0.0,24.99,131540.0,152211.0,213319.0,492997.0,44782.0,29259.0,...,163.333333,24.436684,126.0,195.0,1960.0,173.583333,34.057327,120.0,210.0,2083.0


交叉验证

In [66]:
seed = 2023
kfold = 5

In [67]:
lgb_params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    # 'metric': 'binary_logloss',  # auc??
    'metric': 'auc',
    'n_jobs': 30,
    'learning_rate': 0.05,
    'num_leaves': 2 ** 6,
    'max_depth': 8,
    'tree_learner': 'serial',
    'colsample_bytree': 0.8,
    'subsample_freq': 1,
    'subsample': 0.8,
    'num_boost_round': 3000,
    'max_bin': 255,
    'verbose': -1,
    'nthread' : -1,
    'seed': seed,
    'bagging_seed': seed,
    'feature_fraction_seed': seed,
    'early_stopping_rounds': 100,
    # 'device': 'gpu',  # 设置使用 GPU 加速
    # 'gpu_platform_id': 0,  # 设置 GPU 平台 id
    # 'gpu_device_id': 0  # 设置 GPU 设备 id
}

In [ ]:
# TODO: 训练集最后一列去掉是label，测试集第一列去掉是vid

In [68]:
train = train_fea.iloc[:, 0: -1]
test = test_fea.iloc[:, 1:]
y = train_fea.iloc[:, -1]
test.columns = train.columns 

train.shape, test.shape, y.shape

((19636288, 40), (1584855, 40), (19636288,))

In [69]:
KF = StratifiedKFold(n_splits=kfold, random_state=seed, shuffle=True)
feat_imp_lgb = list()

oof_lgb = np.zeros(len(train))
predictions_lgb = np.zeros((len(test)))
print(train.shape[1])

# 模型训练
for fold_, (trn_idx, val_idx) in enumerate(KF.split(train.values, y.values)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train.iloc[trn_idx], label=y.iloc[trn_idx])
    val_data = lgb.Dataset(train.iloc[val_idx], label=y.iloc[val_idx])
    
    num_round = 3000
    clf = lgb.train(
        lgb_params,
        trn_data,
        num_round,
        valid_sets=[trn_data, val_data],
        verbose_eval=300,
        early_stopping_rounds=100,
    )

    oof_lgb[val_idx] = clf.predict(train.iloc[val_idx], num_iteration=clf.best_iteration)
    predictions_lgb[:] += clf.predict(test, num_iteration=clf.best_iteration) / kfold
    feat_imp_lgb.append(clf.feature_importance())

print("AUC score: {}".format(metrics.roc_auc_score(y, oof_lgb)))
print("F1 score: {}".format(metrics.f1_score(y, [1 if i >= 0.5 else 0 for i in oof_lgb])))
print("Precision score: {}".format(metrics.precision_score(y, [1 if i >= 0.5 else 0 for i in oof_lgb])))
print("Recall score: {}".format(metrics.recall_score(y, [1 if i >= 0.5 else 0 for i in oof_lgb])))

40
fold n°0
[LightGBM] [Warning] num_threads is set with nthread=-1, will be overridden by n_jobs=30. Current value: num_threads=30
Training until validation scores don't improve for 100 rounds
[300]	training's auc: 0.722561	valid_1's auc: 0.719763


# 11111